In [185]:
import zipfile
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
import numpy as np
from sklearn import linear_model, metrics, model_selection
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm



In [186]:
zip = zipfile.ZipFile("data/houses.zip")
zip.extractall("data")
zip.close()

In [187]:
houses = pd.read_csv("data/houses.csv")

In [188]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

In [189]:
houses.describe(include='all')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,2919.000000,2919.000000,2915,2433.000000,2919.000000,2919,198,2919,2919,2917,...,2919.000000,10,571,105,2919.000000,2919.000000,2919.000000,2918,2919,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,...,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,...,NaN,Ex,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,2265,NaN,NaN,2907,120,1859,2622,2916,...,NaN,4,329,95,NaN,NaN,NaN,2525,2402,NaN
mean,1460.000000,57.137718,NaN,69.305795,10168.114080,NaN,NaN,NaN,NaN,NaN,...,2.251799,NaN,NaN,NaN,50.825968,6.213087,2007.792737,NaN,NaN,180921.195890
std,842.787043,42.517628,NaN,23.344905,7886.996359,NaN,NaN,NaN,NaN,NaN,...,35.663946,NaN,NaN,NaN,567.402211,2.714762,1.314964,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN,34900.000000
25%,730.500000,20.000000,NaN,59.000000,7478.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,4.000000,2007.000000,NaN,NaN,129975.000000
50%,1460.000000,50.000000,NaN,68.000000,9453.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN,163000.000000
75%,2189.500000,70.000000,NaN,80.000000,11570.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN,214000.000000


In [190]:
houses.drop(columns=['Id'], inplace=True)

In [191]:
houses.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice        float64
Length: 80, dtype: object

In [192]:
c_float = [x for x in range(len(houses.dtypes)) if houses.dtypes[x] == float]

C:\Users\pabma\AppData\Local\Temp\ipykernel_18544\3693846805.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c_float = [x for x in range(len(houses.dtypes)) if houses.dtypes[x] == float]
C:\Users\pabma\AppData\Local\Temp\ipykernel_18544\3693846805.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c_float = [x for x in range(len(houses.dtypes)) if houses.dtypes[x] == float]
C:\Users\pabma\AppData\Local\Temp\ipykernel_18544\3693846805.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behav

In [193]:
tipos = pd.DataFrame(houses.dtypes)

In [194]:
cols = tipos[tipos[0] == float].reset_index()['index']

In [195]:
cols

0      LotFrontage
1       MasVnrArea
2       BsmtFinSF1
3       BsmtFinSF2
4        BsmtUnfSF
5      TotalBsmtSF
6     BsmtFullBath
7     BsmtHalfBath
8      GarageYrBlt
9       GarageCars
10      GarageArea
11       SalePrice
Name: index, dtype: object

In [196]:
dt = houses[cols]

In [197]:
dt.head()

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea,SalePrice
0,65.0,196.0,706.0,0.0,150.0,856.0,1.0,0.0,2003.0,2.0,548.0,208500.0
1,80.0,0.0,978.0,0.0,284.0,1262.0,0.0,1.0,1976.0,2.0,460.0,181500.0
2,68.0,162.0,486.0,0.0,434.0,920.0,1.0,0.0,2001.0,2.0,608.0,223500.0
3,60.0,0.0,216.0,0.0,540.0,756.0,1.0,0.0,1998.0,3.0,642.0,140000.0
4,84.0,350.0,655.0,0.0,490.0,1145.0,1.0,0.0,2000.0,3.0,836.0,250000.0


In [198]:
dt.dropna(inplace=True)

C:\Users\pabma\AppData\Local\Temp\ipykernel_18544\735218168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt.dropna(inplace=True)


In [199]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1459
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotFrontage   1121 non-null   float64
 1   MasVnrArea    1121 non-null   float64
 2   BsmtFinSF1    1121 non-null   float64
 3   BsmtFinSF2    1121 non-null   float64
 4   BsmtUnfSF     1121 non-null   float64
 5   TotalBsmtSF   1121 non-null   float64
 6   BsmtFullBath  1121 non-null   float64
 7   BsmtHalfBath  1121 non-null   float64
 8   GarageYrBlt   1121 non-null   float64
 9   GarageCars    1121 non-null   float64
 10  GarageArea    1121 non-null   float64
 11  SalePrice     1121 non-null   float64
dtypes: float64(12)
memory usage: 113.9 KB


In [200]:
y = dt['SalePrice']
X = dt.drop(columns=['SalePrice'])

In [201]:
# Esta vez, nuestro modelo serán el 80% de los datos y el test el 20% de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [202]:
X_train

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
460,75.0,110.0,544.0,0.0,288.0,832.0,1.0,0.0,2009.0,2.0,552.0
479,50.0,435.0,168.0,0.0,739.0,907.0,0.0,0.0,1995.0,2.0,672.0
500,21.0,285.0,356.0,0.0,316.0,672.0,0.0,0.0,1973.0,1.0,264.0
683,90.0,215.0,1059.0,0.0,567.0,1626.0,1.0,0.0,2002.0,3.0,702.0
596,60.0,0.0,0.0,0.0,684.0,684.0,0.0,0.0,1930.0,1.0,216.0
...,...,...,...,...,...,...,...,...,...,...,...
598,80.0,459.0,1283.0,147.0,0.0,1430.0,1.0,0.0,1977.0,2.0,621.0
162,95.0,226.0,1201.0,0.0,340.0,1541.0,0.0,0.0,2005.0,2.0,532.0
1364,30.0,0.0,0.0,0.0,600.0,600.0,0.0,0.0,2005.0,2.0,480.0
1426,81.0,448.0,1000.0,0.0,223.0,1223.0,1.0,0.0,1994.0,2.0,525.0


In [203]:
# from sklearn.model_selection import StratifiedShuffleSplit
# split = StratifiedShuffleSplit(n_splits=1, test_size=, random_state=)
# for train_index, test_index in split.split():
#     X_train = .iloc[train_index]
#     X_test = .iloc[test_index]

In [204]:
# Crea una nueva regresión lineal (no uses el mismo nombre que la anterior!!!) usando X_train e y_train
# Realiza la regresión lineal

# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
#X_train = USA_Housing[["Avg. Area Income",	"Avg. Area House Age",	"Avg. Area Number of Rooms", 	"Avg. Area Number of Bedrooms",	"Area Population"]]

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo_ = sm.OLS(endog=y_train, exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo_ = modelo_.fit()
print(modelo_.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     134.1
Date:                Mon, 13 May 2024   Prob (F-statistic):          1.20e-169
Time:                        20:39:40   Log-Likelihood:                -10988.
No. Observations:                 896   AIC:                         2.200e+04
Df Residuals:                     885   BIC:                         2.205e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.033e+06    1.7e+05     -6.068   

In [205]:
modelo_.resid.mean()

-1.9351448697437134e-09

In [206]:
X_test = sm.add_constant(X_test, prepend=True)

In [207]:
preds = modelo_.predict(exog=X_test)

In [208]:
mae = abs(y_test - preds).mean()

In [209]:
mae

37675.08772407305

In [210]:
mae/y_test.mean()

0.19406738766179082

In [211]:
X_test.describe()

,const,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
count,225.0,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000
mean,1.0,69.995556,121.453333,433.733333,38.168889,618.528889,1090.431111,0.457778,0.062222,1981.351111,1.951111,519.293333
std,0.0,23.874579,207.019234,449.511004,133.981502,470.894087,404.316151,0.525463,0.242097,25.601972,0.621283,170.481758
min,1.0,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1910.000000,1.000000,180.000000
25%,1.0,57.000000,0.000000,0.000000,0.000000,240.000000,806.000000,0.000000,0.000000,1966.000000,2.000000,400.000000
50%,1.0,69.000000,0.000000,383.000000,0.000000,528.000000,1075.000000,0.000000,0.000000,1990.000000,2.000000,528.000000
75%,1.0,82.000000,183.000000,705.000000,0.000000,939.000000,1346.000000,1.000000,0.000000,2004.000000,2.000000,626.000000
max,1.0,182.000000,1170.000000,2188.000000,872.000000,1752.000000,2524.000000,2.000000,1.000000,2010.000000,4.000000,936.000000


In [212]:
houses['Street'].unique()
houses

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
2915,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
2916,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
2917,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [213]:
houses['Street'] = houses['Street'].map({'Street':0, 'Pave': 1})

In [214]:
houses['CentralAir'] = houses['CentralAir'].map({'N':0, 'Y': 1})

In [215]:
cols = list(cols.values) + ["Street", "CentralAir"]
dt = houses[cols]

In [216]:
dt.dropna(inplace=True)

C:\Users\pabma\AppData\Local\Temp\ipykernel_18544\735218168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt.dropna(inplace=True)


In [217]:
# Esta vez, nuestro modelo serán el 80% de los datos y el test el 20% de los datos
y = dt['SalePrice']
X = dt.drop(columns=['SalePrice'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [218]:
# Crea una nueva regresión lineal (no uses el mismo nombre que la anterior!!!) usando X_train e y_train
# Realiza la regresión lineal

# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
#X_train = USA_Housing[["Avg. Area Income",	"Avg. Area House Age",	"Avg. Area Number of Rooms", 	"Avg. Area Number of Bedrooms",	"Area Population"]]

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo_ = sm.OLS(endog=y_train, exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo_ = modelo_.fit()
print(modelo_.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.607
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     123.8
Date:                Mon, 13 May 2024   Prob (F-statistic):          2.62e-170
Time:                        20:39:40   Log-Likelihood:                -10973.
No. Observations:                 893   AIC:                         2.197e+04
Df Residuals:                     881   BIC:                         2.203e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
LotFrontage    253.4647     80.459      3.150   

In [219]:
dum = pd.get_dummies(houses['LotShape'], dtype=int)

In [220]:
dt = pd.merge(dt, dum, right_index=True, left_index=True)

In [221]:
dt.drop(columns=['Reg'], inplace=True)

In [222]:
dt.dropna(inplace=True)

In [223]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [224]:
model_r = Ridge()
model_r.fit(X, y)

Ridge()

In [225]:
model_l = Lasso()
model.l.fit(X, y)

AttributeError: 'LinearRegression' object has no attribute 'l'

In [ ]:
model_e = ElasticNet(alpha=0.1, l1_ratio=1)
model_e.fit(X, y)

ElasticNet(alpha=0.1, l1_ratio=1)